In [0]:
# %%capture
# !git clone https://github.com/gem/oq-engine.git

fatal: destination path 'oq-engine' already exists and is not an empty directory.


In [0]:
%%capture
!pip install openquake.engine

In [0]:
import openquake

In [35]:
help(openquake)

Help on package openquake:

NAME
    openquake

PACKAGE CONTENTS
    baselib (package)
    calculators (package)
    commands (package)
    commonlib (package)
    engine (package)
    hazardlib (package)
    hmtk (package)
    qa_tests_data (package)
    risklib (package)
    server (package)

FILE
    (built-in)




In [36]:
help(openquake.hazardlib)

Help on package openquake.hazardlib in openquake:

NAME
    openquake.hazardlib - hazardlib stands for Hazard Library.

PACKAGE CONTENTS
    calc (package)
    const
    contexts
    correlation
    geo (package)
    gsim (package)
    imt
    lt
    mfd (package)
    near_fault
    nrml
    pmf
    probability_map
    scalerel (package)
    shakemap
    shakemapconverter
    site
    site_amplification
    source (package)
    sourceconverter
    sourcewriter
    stats
    tom
    valid

VERSION
    3.9.0

FILE
    /usr/local/lib/python3.6/dist-packages/openquake/hazardlib/__init__.py




In [37]:
help(openquake.engine)

Help on package openquake.engine in openquake:

NAME
    openquake.engine

DESCRIPTION
    OpenQuake is an open-source platform for the calculation of hazard and risk
    impact. It is a project of the Global Earthquake Model, and may be extended
    by other organizations to address additional classes of peril.
    
    For more information, please see the website at http://www.globalquakemodel.org
    This software may be downloaded at http://github.com/gem/openquake
    
    The continuous integration server is at
        https://ci.openquake.org
    
    Up-to-date sphinx documentation is at
        http://docs.openquake.org
    
    This software is licensed under the AGPL license, for more details
    please see the LICENSE file.
    
    Copyright (C) 2010-2020 GEM Foundation
    
    OpenQuake is free software: you can redistribute it and/or modify it
    under the terms of the GNU Affero General Public License as published
    by the Free Software Foundation, either version 3 

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

In [0]:
# time span for stochastic event set generation
time_span = 1000.

# define area source
src = AreaSource(
    source_id='1',
    name='area',
    tectonic_region_type='Active Shallow Crust',
    mfd=TruncatedGRMFD(min_mag=5., max_mag=6.5, bin_width=0.2, a_val=4.45, b_val=0.98),
    rupture_mesh_spacing=2.,
    magnitude_scaling_relationship=WC1994(),
    rupture_aspect_ratio=1.,
    temporal_occurrence_model=PoissonTOM(time_span),
    upper_seismogenic_depth=2.,
    lower_seismogenic_depth=12.,
    nodal_plane_distribution=PMF([(0.125, NodalPlane(strike=0, dip=90, rake=0)),
                                  (0.125, NodalPlane(strike=45, dip=90, rake=0)),
                                  (0.125, NodalPlane(strike=90, dip=90, rake=0)),
                                  (0.125, NodalPlane(strike=135, dip=90, rake=0)),
                                  (0.0625, NodalPlane(strike=0, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=45, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=90, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=135, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=180, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=225, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=270, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=315, dip=50, rake=90))]),
    hypocenter_distribution=PMF([(1, 7.)]),
    polygon=Polygon([Point(133.5, -22.5), Point(133.5, -23.0), Point(130.75, -23.75), Point(130.75, -24.5),
                     Point(133.5, -26.0), Point(133.5, -27.0), Point(130.75, -27.0), Point(128.977, -25.065),
                     Point(128.425, -23.436), Point(126.082, -23.233), Point(125.669, -22.351), Point(125.4, -20.5),
                     Point(125.75, -20.25), Point(126.7, -21.25), Point(128.5, -21.25), Point(129.25, -20.6),
                     Point(130.0, -20.6), Point(130.9, -22.25), Point(133.0, -22.0), Point(133.5, -22.5)]),
    area_discretization=20.
)

In [0]:
# generate two different stochastic event sets from the area source
numpy.random.seed(123)

# generate event and store them in lists
ses1 = list(stochastic_event_set([src]))
ses2 = list(stochastic_event_set([src]))